In [2]:
import pandas as pd
from geofunctions import utils
import geopandas as gpd

In [3]:
gdf_gushdan= gpd.read_file(utils.get_path("processed","adm","gushdan_polygon.geojson"))

In [32]:
import pygeohash as pgh
import geopandas as gpd
from shapely.geometry import Polygon, Point
import pandas as pd

def geohashes_within_area(area_polygon, resolution=7):
    """
    Fills a given area (Polygon) with geohashes of the specified resolution
    and returns them as a GeoDataFrame with geometries.

    :param area_polygon: A shapely Polygon representing the area.
    :param resolution: Geohash resolution (default: 7).
    :return: GeoDataFrame with geohashes and their geometries.
    """
    # Get bounds of the area
    min_lon, min_lat, max_lon, max_lat = area_polygon.bounds

    # Define approximate step size (adjust for finer resolution)
    step_lat = 0.001  # Latitude step, finer for higher resolution
    step_lon = 0.001  # Longitude step, finer for higher resolution

    geohash_list = []

    # Iterate over the bounding box
    lon = min_lon
    while lon < max_lon:
        lat = min_lat
        while lat < max_lat:
            # Generate geohash for the current point
            geohash = pgh.encode(lon, lat, precision=resolution)
            # Decode geohash to get bounding box
            geohash_polygon = utils.geohash_to_polygon(geohash)
            # Add geohash if its polygon intersects with the area
            if area_polygon.intersects(geohash_polygon):
                geohash_list.append({"geohash": geohash, "geometry": geohash_polygon})
            lat += step_lat
        lon += step_lon

    # Convert to GeoDataFrame
    geohash_gdf = gpd.GeoDataFrame(geohash_list, crs="EPSG:4326")
    return geohash_gdf

# Example: Create a sample area polygon
area_polygon = gdf_gushdan.geometry.iloc[0]

# Generate geohashes
geohashes = geohashes_within_area(area_polygon, resolution=7)

# Display results
print(f"Generated {len(geohashes)} geohashes")


Generated 151270 geohashes


In [34]:
geohashes = geohashes.drop_duplicates()

In [41]:
gushdan_census = gpd.read_file(utils.get_path("processed","adm","census_areas_gushdan.geojson"))

In [42]:
geohashes_gushdan = utils.clip_gdf(geohashes,gushdan_census, how='inner')

joined
Original shape: 80182, new shape: 74732


In [44]:
gdf_geohash.merge(geohashes,on='geohash', how='inner').shape

(15826, 17)

In [46]:
geohashes_gushdan ['Main_Function_Txt'].value_counts()

Main_Function_Txt
מגורים         33113
שטח פתוח        6612
תעשיה           3424
מתחם ציבורי     1755
מוסדי            612
Name: count, dtype: int64

In [48]:
geohashes_gushdan.to_file(utils.get_path("processed", "adm", "geohash_stat_area_pairs.geojson")
)